In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


--2019-12-01 14:44:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-01 14:44:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-01 14:44:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:

from keras.models import Model, Sequential, load_model, model_from_config
from keras.layers import Input, concatenate,InputSpec, Activation, Embedding, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional
from keras.engine.topology import Layer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
import keras.backend as K
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import KeyedVectors
from gensim.models import Word2Vec


In [0]:
url = 'https://raw.githubusercontent.com/youko70s/NeuralNets-DeepLearning/master/training_set.tsv'
df = pd.read_csv(url, sep='\t', encoding='ISO-8859-1')


In [0]:
def create_dict(text):
    print("create_dict")
    word_index = {}
    for sentence in text:
        for word in sentence:
            if word not in word_index:
                word_index[word] = len(word_index)
    word_index['unk'] = len(word_index)
    return word_index


def essay_to_index(text, word_index):
    text_index = []
    for essay in text:
        essay_index = []
        for word in essay:
            essay_index.append(word_index[word])
        text_index.append(essay_index)
    return text_index


In [0]:
def get_embeddings_index():
    embeddings_index = {}
    f = open('glove.6B.100d.txt', 'r', encoding='UTF-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index


def embeddings_matrix(embeddings_index, word_index, embedding_dim):
    nb_words = len(word_index)
    embedding_matrix = np.zeros((nb_words + 1, embedding_dim))
    for word, i in word_index.items():
        if i > nb_words:
            continue
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            embedding_matrix[i] = embeddings_vector
    return embedding_matrix




In [0]:
def get_clean_essays(Text):
    clean_essays = []
    for essay in Text:
        clean_essays.append(essay_to_wordlist(essay, remove_stopwords=True))
    return clean_essays


def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
nltk.download('punct')

[nltk_data] Error loading punct: Package 'punct' not found in index


False

In [25]:
Score = df['domain1_score']
Text = df['essay']
word_index = create_dict(essays)

essays = get_clean_essays(Text)

maxlen = 0
for essay in essays:
  if len(essay)>maxlen:
    maxlen = len(essay)

from keras.preprocessing.sequence import pad_sequences

max_essay_length = maxlen
essays_in_index = np.array(essay_to_index(essays, word_index))
essays_in_index = pad_sequences(essays_in_index, maxlen=max_essay_length)


embeddings_index = get_embeddings_index()


create_dict


In [31]:
len(embeddings_index)

400000

In [0]:
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, 100))
for word, i in word_index.items():
    if i > nb_words:
        continue
    embeddings_vector = embeddings_index.get(word)
    if embeddings_vector is not None:
        embedding_matrix[i] = embeddings_vector

In [16]:
maxlen

524

In [0]:
def get_model():
    print('Build model...')
    #model = Sequential()
    text_input = Input(shape=(max_essay_length, ), name='text_input')
    # feature_input = Input(shape=(num_feature, ), name='feature_input')
    # model = Sequential()
    
    embedding_layer = Embedding(input_dim=len(word_index) + 1,
                                output_dim=100,
                                weights=[embedding_matrix],
                                input_length=max_essay_length,
                                trainable=True,
                                dropout=0)
    embedding = embedding_layer(text_input)
    # model.add(embedding)
    #lstm_out = LSTM(64, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)(embedding)
    lstm_out = LSTM(64, dropout=0.1, recurrent_dropout=0.1, return_sequences=False)(embedding)
    #model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=False))
    #att = AttentionWithContext(lstm_out)
    #att = Addition(att)
    output = Dense(1, activation='relu')(lstm_out)

    #add attention
    #model.add(AttentionWithContext())
    #model.add(Addition())
    #model.add(Dense(1, activation='sigmoid'))
    model = Model(inputs=text_input, outputs=output)
    model.summary()
    return model

In [98]:
get_model()

Build model...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  del sys.path[0]


Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 524)               0         
_________________________________________________________________
embedding_36 (Embedding)     (None, 524, 100)          3785600   
_________________________________________________________________
lstm_47 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 65        
Total params: 3,827,905
Trainable params: 3,827,905
Non-trainable params: 0
_________________________________________________________________


In [0]:
results = []
score_predict_list = []
count = 1
cv = KFold(n_splits=3, shuffle=True)
for train, test in cv.split(essays_in_index):
    text_train, text_test = essays_in_index[train], essays_in_index[test]
    label_train, label_test = Score.iloc[train], Score.iloc[test]

    model = get_model()
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    history = model.fit(text_train, label_train, batch_size=16, epochs=20)

    score_predict = model.predict(text_test)
    score_predict = np.around(score_predict)
    result = cohen_kappa_score(label_test.values, score_predict, weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1

print("Average Kappa score after a 5-fold cross validation: ", np.around(np.array(results).mean(), decimals=4))


Build model...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  del sys.path[0]


Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 524)               0         
_________________________________________________________________
embedding_39 (Embedding)     (None, 524, 100)          3785600   
_________________________________________________________________
lstm_50 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 65        
Total params: 3,827,905
Trainable params: 3,827,905
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
8650/8650 [==============================] - 273s 32ms/step - loss: 40.9745 - mean_absolute_error: 3.3963
Epoch 2/20
8304/8650 [===========================>..] - ETA: 10s - loss: 14.1083 - mean_absolute_error: 1.9543